# Programming Task Description

## Programming Task: Implementing a Character-Level GPT Model

### Introduction
In this task, you will create a Python script using PyTorch to implement a simplified GPT (Generative Pre-trained Transformer) model for character-level language modeling. The model will be trained on the text in input.txt to predict the next character in a sequence and generate new text based on a given context. The architecture follows the decoder part of the transformer model from the "Attention is All You Need" paper by Vaswani et al., focusing on masked multi-head self-attention to ensure predictions depend only on previous positions.

## Task Description
### Your goal is to write a Python jupyter notebook that:

1. Reads and processes the text from input.txt.
2. Implements a decoder-only transformer model with manual attention mechanisms.
3. Trains the model on the processed data.
4. Generates new text using the trained model.

You will use PyTorch and implement the attention mechanism from scratch, following the decoder structure outlined in the "Attention is All You Need" paper.

### Step-by-step Guide

1. Data Preparation
* Read all text from input.txt using UTF-8 encoding.
* Create a sorted list of unique characters (vocabulary) from the text.
* Build two dictionaries:
    * stoi: Maps characters to integers (e.g., 'a' -> 0).
    * itos: Maps integers to characters (e.g., 0 -> 'a').
* Define functions:
    * encode(s): Converts a string to a list of integers using stoi.
    * decode(l): Converts a list of integers to a string using itos.
* Encode the entire text into a tensor of integers using torch.tensor.
* Split the data: 90% for training, 10% for validation.

2. Data Loading
* Implement a function get_batch(split):
    * Input: split is either 'train' or 'val'.
    * Select the appropriate dataset (training or validation).
    * Randomly sample batch_size starting indices, ensuring each sequence fits within block_size.
* Return:
    * x: A tensor of shape (batch_size, block_size) with input sequences.
    * y: A tensor of shape (batch_size, block_size) with target sequences (shifted by one position).
* Move tensors to the device (CPU or GPU).

3. Model Architecture
* Implement the following components in a decoder-only transformer:
    * Embedding Layers:
        * Token embedding: nn.Embedding(vocab_size, n_embd) for character indices.
        * Position embedding: nn.Embedding(block_size, n_embd) for positions 0 to block_size-1.
    * Transformer Blocks:
        * Each block includes:
            * Masked Multi-Head Self-Attention:
                * Implement manually (do not use nn.MultiheadAttention).
                * For each head:
                    * Linear layers for queries (Q), keys (K), and values (V).
                    * Scaled dot-product attention: attention = softmax((Q @ K.T) / sqrt(d_k)) @ V.
                    * Mask future positions with a lower triangular matrix (e.g., tril) by setting future weights to -inf before softmax.
                * Concatenate heads and apply a projection layer.
            * Feed-Forward Network: nn.Linear(n_embd, 4 * n_embd) → ReLU → nn.Linear(4 * n_embd, n_embd).
            * Layer Normalization: Apply nn.LayerNorm(n_embd) before each sub-layer (pre-norm).
            * Residual Connections: Add input to output of each sub-layer.
        * Use n_layer blocks in sequence.
    * Final Layers:
        * nn.LayerNorm(n_embd) for final normalization.
        * nn.Linear(n_embd, vocab_size) to produce logits.
* Define a GPTLanguageModel class with:
    * forward(idx, targets=None): Computes logits and loss (if targets provided).
    * generate(idx, max_new_tokens): Autoregressively generates new tokens.

4. Training
* Use the AdamW optimizer with learning_rate = 3e-4.
* Train for max_iters = 5000 iterations.
* Estimate and print training and validation losses:
* Compute loss using F.cross_entropy on flattened logits and targets.

5. Text Generation
* Implement generate(idx, max_new_tokens):
    * Start with an initial context idx (shape (B, T)).
    * For max_new_tokens steps:
        * Crop idx to the last block_size tokens.
        * Get logits from forward.
        * Apply softmax to the last time step’s logits to get probabilities.
        * Sample the next token using torch.multinomial.
        * Append the sampled token to idx.
    * Return the extended sequence.

### Hyperparameters
Use these values:

* batch_size = 64
* block_size = 256
* n_embd = 384
* n_head = 6
* n_layer = 6
* dropout = 0.2
* learning_rate = 3e-4
* max_iters = 5000

### Understanding the Decoder
The "Attention is All You Need" paper describes a transformer with an encoder and decoder. For this task, you focus on the decoder-only architecture used in GPT:

* Masked Self-Attention: Ensures the model only attends to previous positions in the sequence, making it autoregressive. This is achieved by masking future tokens in the attention computation, as shown below:

$Attention (Q, K, V) = softmax((Q@K.T)/sqrt(d_{k}) + mask) @V$

where $mask$ sets future positions to $-inf$

* Decoder Role: In the original paper, the decoder generates output sequences while attending to the encoder’s output. Here, without an encoder, it uses self-attention on the input sequence alone, predicting the next token step-by-step.

### Additional Notes
* Manual Attention: Implement attention from scratch to understand its mechanics (no pre-built PyTorch modules).
* Masking: Use a lower triangular matrix (e.g., torch.tril) to mask future positions.
* Device Handling: Set device = 'cuda' if torch.cuda.is_available() else 'cpu' and move tensors/models accordingly.
* Dropout: Apply nn.Dropout(dropout) in attention and feed-forward layers for regularization.

### Deliverables
A Python script that:
* Implements all steps above.
* Prints training and validation losses every 500/100? iterations (up to you).
* Generates and prints a 500-character sample after training.

### Evaluation Criteria
* Correct data preparation and batch loading.
* Accurate implementation of the transformer model, especially masked self-attention.
* Successful training with decreasing loss.
* Generation of coherent (for character-level) text.

### Step 1: Import libraries and prepare data


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Step 2: Load and process text data

In [2]:
# open and read the input text
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# get sorted list of unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create mappings
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# encoding and decoding functions
def encode(s): return [stoi[c] for c in s]
def decode(l): return ''.join([itos[i] for i in l])

# convert full dataset to tensor
data = torch.tensor(encode(text), dtype=torch.long)

# split into train and val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


### Step 3: Define hyperparameters and data loader

In [3]:
batch_size = 64
block_size = 256
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
learning_rate = 3e-4
max_iters = 5000

# get a batch of input and target sequences
def get_batch(split):
    d = train_data if split == "train" else val_data
    ix = torch.randint(0, len(d) - block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size] for i in ix])
    y = torch.stack([d[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)


### Step 4: Build model components

In [4]:
# one attention head
class Head(nn.Module):
    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # Compute scaled dot-product attention
        weight = q @ k.transpose(-2, -1) / (C ** 0.5)
        weight = weight.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # mask future positions
        weight = F.softmax(weight, dim=-1)
        weight = self.dropout(weight)
        v = self.value(x)
        out = weight @ v
        return out # return weighted sum of values


Multi-head Attention

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concatenate heads
        out = self.proj(out)
        out = self.dropout(out)
        return out # project back to embedding size

Feedforward Network

In [6]:
class FeedForward(nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

Transformer Block

In [7]:
# transformer block = attention + ff + residual + norm
class Block(nn.Module):
    def __init__(self):
        super(Block, self).__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # attention + residual
        x = x + self.ffwd(self.ln2(x)) # feedforward + residual
        return x

### Step 5: Final Model

In [8]:
class GPT(nn.Module):
    def __init__(self):
        super(GPT, self).__init__()
        # Embedding for tokens and positions
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)

        # Stack of Transformer blocks
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) # final linear layer for logits

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)  # (B, T, n_embd)
        pos_emb = self.pos_emb(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb # add token + position embeddings
        x = self.blocks(x) # apply Transformer blocks
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        # Compute cross-entropy loss if targets are provided
        if targets is None:
            return logits, None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss

    # text generation
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] # keep only the latest block_size tokens
            logits, _ = self(idx_cond)  # forward pass
            logits = logits[:, -1, :] # focus on last token only
            probs = F.softmax(logits, dim=-1) # convert to probabilities
            next_token = torch.multinomial(probs, num_samples=1) # sample next toke
            idx = torch.cat((idx, next_token), dim=1) # append to sequence
        return idx

### Step 6: Create model and train

In [9]:
# Create model and optimizer
model = GPT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for step in range(max_iters):
    model.train()
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Print loss every 500 steps and check validation
    if step % 500 == 0:
        print(f"Step {step}:")
        print(f"   Train Loss = {loss.item():.4f}")
        model.eval()
        with torch.no_grad():
            xb_val, yb_val = get_batch('val')
            _, val_loss = model(xb_val, yb_val)
            print(f"   Val Loss   = {val_loss.item():.4f}")


Step 0:
   Train Loss = 4.3798
   Val Loss   = 3.6481
Step 500:
   Train Loss = 2.0748
   Val Loss   = 2.0668
Step 1000:
   Train Loss = 1.6997
   Val Loss   = 1.7245
Step 1500:
   Train Loss = 1.5302
   Val Loss   = 1.6403
Step 2000:
   Train Loss = 1.4490
   Val Loss   = 1.6107
Step 2500:
   Train Loss = 1.3245
   Val Loss   = 1.5408
Step 3000:
   Train Loss = 1.2918
   Val Loss   = 1.5076
Step 3500:
   Train Loss = 1.2876
   Val Loss   = 1.4755
Step 4000:
   Train Loss = 1.2428
   Val Loss   = 1.5397
Step 4500:
   Train Loss = 1.2130
   Val Loss   = 1.5460


### Step 7: Generate output

In [10]:
# Start with an empty context (just one token)
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Generate 500 characters of text
output = model.generate(context, max_new_tokens=500)

# Decode and print the generated character-level text
print(decode(output[0].tolist()))




LUCIO:
Here's brother, either you are a poor man.

LUCIO:
A sea tormors; within him that can prove him be crown'd
this counted by the pettion of the purposition,
A sin chief what obtain has accurse;
And Besubashamas, are they like a mand off.

bERBETH:
The grave saves on him.

RICHARD:
Suspeiant one that time was to me too,
That I am beat an intenedly fuce.

NORTHUMBERLAND:
O, like these tears, as though to noble.
So Kill hate I had mark
Afrown: what, the clouds? his mother own own of the lives
